In [1]:
import importlib
import wrf_management.project_global_constants as gc
import wrf_management.ungrib as un
import os
import pandas as pd
import subprocess as su
import wrf_management.run_utilities as ru
import wrf_management.metgrid as me

In [2]:
print(gc.RUN_NAME)
job = 'metgrid'
ungrib_source_dirs = ['ungrib_surf', 'ungrib_press']
avg_pref = 'ungrib_avgtsfc'
real = True
hours=24
LIST_S_LINKS = [
    'metgrid.exe',
    #     'link_grib.csh',
    'metgrid'
]

LIST_H_LINKS = [
    #     'Vtable',
    'env_WRFv4.bash'
]

run_2019_02_20


In [3]:
# con = sq.connect(gc.PATH_DB)
gc.PATH_DB

'/homeappl/home/aliagadi/saltena_2018/wrf_management/wrf_management/db_folder/wrf_man.sqlite'

In [4]:
importlib.reload(un)
importlib.reload(ru)
run_row = ru.get_run_row()
print(run_row)

index                             1
run_name             run_2019_02_20
geogrid_path                geogrid
config_path          run_2019_02_20
data_path       runs/run_2019_02_20
Name: 0, dtype: object


In [5]:
run_path = os.path.join(gc.PATH_DATA, run_row.data_path)
print(run_path)

/wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/run_2019_02_20


In [6]:
job_row = ru.get_next_row(job=job)
print(job_row)

date              2017-12-02 00:00:00
ungrib_surf                       100
ungrib_press                      100
ungrib_lake                         0
ungrib_avgtsfc                    100
metgrid                             0
real                                0
Name: 0, dtype: object


In [7]:
if real:
    ru.update_run_table(val=job_row[job] + 1,
                        job=job,
                        date=job_row['date']
                        )

job_path = ru.getmk_job_path(run_row, job_row, job)
print(job_path)

/wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/run_2019_02_20/2017_12_02/metgrid


In [8]:
conf_path = ru.get_conf_path(run_row)
print(conf_path)

/homeappl/home/aliagadi/saltena_2018/wrf_management/wrf_management/config_dir/run_2019_02_20


In [9]:
importlib.reload(me)
name_list = me.skim_namelist_copy_metgrid(
    conf_path, job_path, date=job_row.date, prefix=job, hours=hours
)
pd.DataFrame(name_list)

,share,metgrid
constants_name,NaN,TAVGSFC
end_date,"[2017-12-03_00:00:00, 2017-12-03_00:00:00, 201...",NaN
fg_name,NaN,"[ungrib_surf, ungrib_press]"
interval_seconds,21600,NaN
io_form_geogrid,2,NaN
io_form_metgrid,NaN,2
max_dom,4,NaN
start_date,"[2017-12-02_00:00:00, 2017-12-02_00:00:00, 201...",NaN
wrf_core,ARW,NaN


In [10]:

importlib.reload(un)
un.link_grub_files(ungrib_prefixes=ungrib_source_dirs, job_path=job_path)

In [11]:
importlib.reload(me)
me.get_geo_files(
    run_path=run_path,
    geo_rel_path=run_row.geogrid_path)
me.link_geo_files(
    geo_rel_path=run_row.geogrid_path,
    run_path=run_path,
    job_path=job_path
)

In [12]:
importlib.reload(me)
me.link_avg_file(prefix=avg_pref, job_path=job_path)

In [13]:
# if gc.ID=='taito_login':
ru.copy_hard_links(conf_path, job_path, LIST_H_LINKS)
ru.copy_soft_links(
    os.path.join(gc.PATH_WPS, ''),
    job_path, LIST_S_LINKS)

env_WRFv4.bash
unilinking
metgrid.exe
unilinking
metgrid


In [14]:
run_script = \
    """#!/bin/bash
    cd {job_path}
    source ./env_WRFv4.bash 
    ./metgrid.exe
    exit $?
    """.format(job_path=job_path)
print(run_script)
bs_path = os.path.join(job_path, 'run_me.sh')
bs_file = open(bs_path, 'w')
bs_file.write(run_script)
bs_file.close()

#!/bin/bash
    cd /wrk/aliagadi/DONOTREMOVE/wrf_management_data/runs/run_2019_02_20/2017_12_02/metgrid
    source ./env_WRFv4.bash 
    ./metgrid.exe
    exit $?
    


In [15]:
if gc.ID == 'taito_login':
    res = su.run(['/bin/bash', bs_path], stdout=su.PIPE, stderr=su.PIPE)

if gc.ID == 'taito_login' and res.returncode == 0:
    un.update_run_table(val=100,
                        job=job,
                        date=job_row['date']
                        )